In [1]:
import numpy as np
import torch
import torch.nn as nn # neural network
import torch.optim as optim
import torchvision
from torchvision import datasets # CIFAR10, CIFAR100
from torchvision import transforms # augmentation lib
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import time
import os

In [2]:
# Device confg -> GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
# def data_loader(data_dir, batch_size, random_seed=42, shuffle=True, valid_size=0.1, test=False):
#     normalize = transforms.Normalize(
#         mean = [0.4914, 0.4822, 0.4465], # x - mean / std
#         std = [0.2023, 0.1994, 0.2010]
#     )
# # define transforms
#     transform = transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         normalize
#     ])

#     if test:
#         dataset = datasets.CIFAR10(
#             root = data_dir, train=False, download=True, transform=transform)
        
#         data_loader = torch.utils.data.DataLoader(
#             dataset, batch_size=batch_size, shuffle=shuffle
#         )
#         return data_loader

# # load the dataset
#     train_dataset = datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
#     valid_dataset = datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
    
#     num_train = len(train_dataset)
#     indices = list(range(num_train))
#     split = int(np.floor(valid_size * num_train))

#     if shuffle:
#         np.random.seed(random_seed)
#         np.random.shuffle(indices)
    
#     train_idx, valid_idx = indices[split:], indices[:split]
#     train_sampler = SubsetRandomSampler(train_idx)
#     valid_sampler = SubsetRandomSampler(valid_idx)

#     train_loader = torch.utils.data.DataLoader(
#         train_dataset, batch_size=batch_size, sampler=train_sampler 
#     )
    
#     valid_loader = torch.utils.data.DataLoader(
#         valid_dataset, batch_size=batch_size, sampler=valid_sampler
#     )
#     return (train_loader, valid_loader)


In [4]:
def get_dataloaders_cifar10(batch_size, num_workers=0,
                            validation_fraction=None,
                            train_transforms=None,
                            test_transforms=None):

    if train_transforms is None:
        train_transforms = transforms.ToTensor()

    if test_transforms is None:
        test_transforms = transforms.ToTensor()

    train_dataset = datasets.CIFAR10(root='/kaggle/working/data',
                                     train=True,
                                     transform=train_transforms,
                                     download=True)

    valid_dataset = datasets.CIFAR10(root='/kaggle/working/data',
                                     train=True,
                                     transform=test_transforms)

    test_dataset = datasets.CIFAR10(root='/kaggle/working/data',
                                    train=False,
                                    transform=test_transforms)

    if validation_fraction is not None:
        num = int(validation_fraction * 50000)
        train_indices = torch.arange(0, 50000 - num)
        valid_indices = torch.arange(50000 - num, 50000)

        train_sampler = SubsetRandomSampler(train_indices)
        valid_sampler = SubsetRandomSampler(valid_indices)

        valid_loader = DataLoader(dataset=valid_dataset,
                                  batch_size=batch_size,
                                  num_workers=num_workers,
                                  sampler=valid_sampler)

        train_loader = DataLoader(dataset=train_dataset,
                                  batch_size=batch_size,
                                  num_workers=num_workers,
                                  drop_last=True,
                                  sampler=train_sampler)

    else:
        train_loader = DataLoader(dataset=train_dataset,
                                  batch_size=batch_size,
                                  num_workers=num_workers,
                                  drop_last=True,
                                  shuffle=True)

    test_loader = DataLoader(dataset=test_dataset,
                             batch_size=batch_size,
                             num_workers=num_workers,
                             shuffle=False)

    if validation_fraction is None:
        return train_loader, test_loader
    else:
        return train_loader, valid_loader, test_loader

In [5]:
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),  # 랜덤 수평 뒤집기
    torchvision.transforms.RandomCrop(32, padding=4),  # 랜덤 크롭 (패딩 포함)
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),                
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_loader, valid_loader, test_loader = get_dataloaders_cifar10(
    batch_size=64,
    validation_fraction=0.1,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    num_workers=2)

100%|██████████| 170498071/170498071 [00:06<00:00, 28284330.84it/s]


Extracting /kaggle/working/data/cifar-10-python.tar.gz to /kaggle/working/data


In [6]:
# train_loader, valid_loader = data_loader(data_dir='/kaggle/working/data', batch_size=64)

In [7]:
# test_loader = data_loader(data_dir='/kaggle/working/data', batch_size=64, test=True)

In [8]:
import torch
import torch.nn as nn

class VGG16(nn.Module):
    def __init__(self, num_classes=100):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU()
        )
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU()
        )
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected layer
        self.fc = nn.Sequential(
            nn.Linear(7*7*512, 4096),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(4096, num_classes)
        )
        
        for m in self.modules():
            if isinstance(m, torch.torch.nn.Conv2d) or isinstance(m, torch.torch.nn.Linear):
                torch.nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    m.bias.detach().zero_()
                    
        self.avgpool = torch.nn.AdaptiveAvgPool2d((7, 7))        

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)
        x = self.layer9(x)
        x = self.layer10(x)
        x = self.layer11(x)
        x = self.layer12(x)
        x = self.layer13(x)
        x = self.avgpool(x)
        x = x.reshape(x.size(0), -1)  # flatten (batch_size, num_channels * height * width)
        x = self.fc(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [9]:
!pip install torchinfo  # Install the missing module
from torchinfo import summary  # Import after installation

# Define a sample input tensor
features = torch.randn(1, 3, 224, 224)  # Assuming your model expects input of shape (batch, channels, height, width)

# Move the model and input to the same device (if using GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG16(num_classes=10).to(device)
model = model.to(device)
features = features.to(device)

summary(model, input_data=features, col_names=("input_size", "output_size", "num_params"))  # Now this should work

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
VGG16                                    [1, 3, 224, 224]          [1, 10]                   --
├─Sequential: 1-1                        [1, 3, 224, 224]          [1, 64, 224, 224]         --
│    └─Conv2d: 2-1                       [1, 3, 224, 224]          [1, 64, 224, 224]         1,792
│    └─BatchNorm2d: 2-2                  [1, 64, 224, 224]         [1, 64, 224, 224]         128
│    └─ReLU: 2-3                         [1, 64, 224, 224]         [1, 64, 224, 224]         --
├─Sequential: 1-2                        [1, 64, 224, 224]         [1, 64, 112, 112]         --
│    └─Conv2d: 2-4                       [1, 64, 224, 224]         [1, 64, 224, 224]         36,928
│    └─BatchNorm2d: 2-5                  [1, 64, 224, 224]         [1, 64, 224, 224]         128
│    └─ReLU: 2-6                         [1, 64, 224, 224]         [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-7     

In [10]:
total_step = len(train_loader)
total_step

703

In [20]:
def plot_training_loss(minibatch_loss_list, num_epochs, iter_per_epoch,
                       results_dir=None, averaging_iterations=100):

    plt.figure()
    ax1 = plt.subplot(1, 1, 1)
    ax1.plot(range(len(minibatch_loss_list)),
             (minibatch_loss_list), label='Minibatch Loss')

    if len(minibatch_loss_list) > 1000:
        ax1.set_ylim([
            0, np.max(minibatch_loss_list[1000:])*1.5
            ])
    ax1.set_xlabel('Iterations')
    ax1.set_ylabel('Loss')

    ax1.plot(np.convolve(minibatch_loss_list,
                         np.ones(averaging_iterations,)/averaging_iterations,
                         mode='valid'),
             label='Running Average')
    ax1.legend()

    ###################
    # Set scond x-axis
    ax2 = ax1.twiny()
    newlabel = list(range(num_epochs+1))

    newpos = [e*iter_per_epoch for e in newlabel]

    ax2.set_xticks(newpos[::10])
    ax2.set_xticklabels(newlabel[::10])

    ax2.xaxis.set_ticks_position('bottom')
    ax2.xaxis.set_label_position('bottom')
    ax2.spines['bottom'].set_position(('outward', 45))
    ax2.set_xlabel('Epochs')
    ax2.set_xlim(ax1.get_xlim())
    ###################

    plt.tight_layout()

    if results_dir is not None:
        image_path = os.path.join(results_dir, '/kaggle/working/data/plot_training_loss.pdf')
        plt.savefig(image_path)

def plot_accuracy(train_acc_list, valid_acc_list, results_dir):

    num_epochs = len(train_acc_list)

    plt.plot(np.arange(1, num_epochs+1),
             train_acc_list, label='Training')
    plt.plot(np.arange(1, num_epochs+1),
             valid_acc_list, label='Validation')

    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()

    if results_dir is not None:
        image_path = os.path.join(
            results_dir, '/kaggle/working/data/plot_acc_training_validation.pdf')
        plt.savefig(image_path)

In [12]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [13]:
def train_model(model, num_epochs, train_loader,
                valid_loader, test_loader, optimizer,
                device, logging_interval=50,
                scheduler=None,
                scheduler_on='valid_acc',
                start_epoch=0,
                checkpoint_path='/kaggle/working/data/checkpoint1.pth.tar'):

    start_time = time.time()
    minibatch_loss_list, train_acc_list, valid_acc_list = [], [], []
    train_loss_list, valid_loss_list = [], []

    for epoch in range(start_epoch, num_epochs):

        model.train()
        running_loss = 0.0
        for batch_idx, (features, targets) in enumerate(train_loader):

            features = features.to(device)
            targets = targets.to(device)

            # ## FORWARD AND BACK PROP
            logits = model(features)
            loss = torch.nn.functional.cross_entropy(logits, targets)
            optimizer.zero_grad()
            loss.backward()

            # ## UPDATE MODEL PARAMETERS
            optimizer.step()

            # ## LOGGING
            minibatch_loss_list.append(loss.item())
            running_loss += loss.item() * features.size(0)
            if not batch_idx % logging_interval:
                print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                      f'| Batch {batch_idx:04d}/{len(train_loader):04d} '
                      f'| Loss: {loss:.4f}')

        train_loss = running_loss / len(train_loader.dataset)
        train_loss_list.append(train_loss)

        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():  # save memory during inference
            train_acc = compute_accuracy(model, train_loader, device=device)
            valid_acc = compute_accuracy(model, valid_loader, device=device)
            
            for features, targets in valid_loader:
                features, targets = features.to(device), targets.to(device)
                logits = model(features)
                loss = torch.nn.functional.cross_entropy(logits, targets)
                running_val_loss += loss.item() * features.size(0)
                
            valid_loss = running_val_loss / len(valid_loader.dataset)
            valid_loss_list.append(valid_loss)

            print(f'Epoch: {epoch+1:03d}/{num_epochs:03d} '
                  f'| Train Loss: {train_loss:.4f} | Validation Loss: {valid_loss:.4f} '
                  f'| Train: {train_acc :.2f}% | Validation: {valid_acc :.2f}%')
            train_acc_list.append(train_acc.item())
            valid_acc_list.append(valid_acc.item())

        elapsed = (time.time() - start_time)/60
        print(f'Time elapsed: {elapsed:.2f} min')

        if scheduler is not None:
            if scheduler_on == 'valid_acc':
                scheduler.step(valid_acc_list[-1])
            elif scheduler_on == 'minibatch_loss':
                scheduler.step(minibatch_loss_list[-1])
            else:
                raise ValueError(f'Invalid `scheduler_on` choice.')

        # Save checkpoint
        save_checkpoint({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, filename=checkpoint_path)

    elapsed = (time.time() - start_time)/60
    print(f'Total Training Time: {elapsed:.2f} min')

    test_acc = compute_accuracy(model, test_loader, device=device)
    print(f'Test accuracy {test_acc :.2f}%')

    return minibatch_loss_list, train_acc_list, valid_acc_list, train_loss_list, valid_loss_list


In [26]:
def plot_epoch_losses(train_loss_list, valid_loss_list):
    epochs = range(1, len(train_loss_list) + 1)
    plt.plot(epochs, train_loss_list, label='Train Loss')
    plt.plot(epochs, valid_loss_list, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Train and Validation Loss per Epoch')
    plt.legend()
    plt.grid(True)
    plt.show()
    plt.savefig(os.path.join(result_dir, 'plot_accuracy.pdf'))

In [15]:
# hyper parameters
num_classes = 10
num_epochs = 30
batch_size = 64
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG16(num_classes).to(device)

# loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       mode='max',
                                                       verbose=True)
# checkpoints
def save_checkpoint(state, filename='/kaggle/working/data/checkpoint1.pth.tar'):
    torch.save(state, filename)

# load checkpoints
def load_checkpoint(filename):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    return epoch
    
start_epoch = 0
checkpoint_path = '/kaggle/working/data/checkpoint1.pth.tar'
try:
    start_epoch = load_checkpoint(checkpoint_path)
    print(f'Checkpoint loaded, starting from epoch {start_epoch + 1}')
except FileNotFoundError:
    print('No checkpoint found, starting from scratch')

# train and evaluation
minibatch_loss_list, train_acc_list, valid_acc_list = train_model(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    num_epochs=num_epochs,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    device=device,
    scheduler_on='valid_acc',
    start_epoch=start_epoch,
    checkpoint_path=checkpoint_path,
    logging_interval=100
)
plot_training_loss(minibatch_loss_list=minibatch_loss_list,
                   num_epochs=num_epochs,
                   iter_per_epoch=len(train_loader),
                   results_dir='None',
                   averaging_iterations=200)
plt.show()

plot_accuracy(train_acc_list=train_acc_list, 
              valid_acc_list=valid_acc_list,
              result_dri='None')
plt.ylim([60, 100])
plt.show()

plot_epoch_losses(train_loss_list, valid_loss_list)

No checkpoint found, starting from scratch
Epoch: 001/030 | Batch 0000/0703 | Loss: 5.3999
Epoch: 001/030 | Batch 0100/0703 | Loss: 2.6478
Epoch: 001/030 | Batch 0200/0703 | Loss: 2.2804
Epoch: 001/030 | Batch 0300/0703 | Loss: 2.2120
Epoch: 001/030 | Batch 0400/0703 | Loss: 2.2044
Epoch: 001/030 | Batch 0500/0703 | Loss: 2.1252
Epoch: 001/030 | Batch 0600/0703 | Loss: 2.2128
Epoch: 001/030 | Batch 0700/0703 | Loss: 2.1666
Epoch: 001/030 | Train Loss: 3.5026 | Validation Loss: 0.2056 | Train: 23.54% | Validation: 24.44%
Time elapsed: 8.93 min
Epoch: 002/030 | Batch 0000/0703 | Loss: 2.1129
Epoch: 002/030 | Batch 0100/0703 | Loss: 2.1739
Epoch: 002/030 | Batch 0200/0703 | Loss: 2.2772
Epoch: 002/030 | Batch 0300/0703 | Loss: 2.4276
Epoch: 002/030 | Batch 0400/0703 | Loss: 1.9112
Epoch: 002/030 | Batch 0500/0703 | Loss: 2.0038
Epoch: 002/030 | Batch 0600/0703 | Loss: 1.9091
Epoch: 002/030 | Batch 0700/0703 | Loss: 1.9954
Epoch: 002/030 | Train Loss: 1.8330 | Validation Loss: 0.1879 | Tra

ValueError: too many values to unpack (expected 3)

In [ ]:
# # testing
# with torch.no_grad():
#     correct = 0
#     total = 0
#     for images, labels in test_loader:
#         images = images.to(device)
#         labels = labels.to(device)
#         outputs = model(images)

#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#         del images, labels, outputs

#     accuracy = (correct / total) * 100
#     print('Accuracy of the network on the {} test images: {:.6f} %'.format(total, accuracy))

In [ ]:
def save_checkpoint(model, optimizer, epoch, path='/kaggle/working/data/checkpoint.pth'):
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(state, path)
    print(f'Model saved to {path}')

# save final model
save_checkpoint(model, optimizer, num_epochs)

# 런타임 해제 코드
import os
os.system('kill -9 -1')